In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled)


In [2]:
%matplotlib inline
import utils_modified; reload(utils_modified)
from utils_modified import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
import numpy as np
import random
import sys

In [4]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Embedding, Activation, LSTM, merge, Flatten, Dropout, Lambda
from keras.models import Model, Sequential
from keras.engine.topology import Merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.convolutional import *
from keras.utils.data_utils import get_file

In [64]:
# https://keras.io/getting-started/sequential-model-guide/

In [158]:
# https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()#.lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

corpus length: 600901
total chars: 85
nb sequences: 200287


In [159]:
lag = 4

In [246]:
idx = [char_indices[c] for c in text]

Xs = []
for l in range(lag):
    cdat = [idx[i+l] for i in xrange(0, len(idx)-1-lag, lag)]
    X = np.stack(cdat[:-3])
    Xs.append(X)

cdat = [idx[i+(l+1)] for i in xrange(0, len(idx)-1-lag, lag)]
Y = np.stack(cdat[:-3])

In [162]:
inputs = [Input(shape=(1,), dtype='int64') for i in range(lag)]

E = Embedding(output_dim=50, input_dim=len(chars), input_length=1)

Di = Dense(50, activation='relu')
Dh = Dense(50, activation='relu', init='identity')

#hidden = ... CONSTANT ZERO TENSOR IN KERAS ?

e = Di(Flatten()(E(inputs[0])))
hidden = e
for i in range(1,lag):
    e = Di(Flatten()(E(inputs[i])))
    hidden = merge([e, Dh(hidden)], mode='sum')
predictions = Dense(len(chars), activation='softmax')(hidden)

model = Model(input=inputs, output=predictions)

model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [171]:
model.optimizer.lr = 1e-3
model.fit(Xs, to_categorical(Y,len(chars)), nb_epoch=10)

Epoch 1/1
150221/150221 [==============================] - 18s - loss: 2.0809 - acc: 0.4002    


In [243]:
def show_top_next(inp):
    idxs = [char_indices[c] for c in inp][-lag:]
    ps = model.predict([np.array([i]) for i in idxs])
    probas = ps[0]/np.sum(ps)
    for i in range(5):
        index = np.random.choice(range(len(chars)), size=None, replace=True, p=probas)
        print(inp+chars[index])
    print('')

In [245]:
show_top_next('hersel')
show_top_next('himsel')
show_top_next('mora')
show_top_next('moral')
show_top_next('morali')

herseld
herself
herself
hersel,
hersel 

himself
himseld
himself
himself
himself

mora 
morau
morat
mora 
mora 

morall
morali
moraln
moral 
moraly

moralit
morali 
moralie
moralit
moralid

